## Locating regulatory motifs / transcription factor binding sites

### Regulatory motifs are not conserved, and may vary at some positions. Thus, we must develop 'motif finding' algorithms.
> ### In contrast to a *DnaA* box, which is a pattern that clumps, or appears frequently, within a relatively short interval of the genome, a regulatory motif is a pattern that appears at least once (perhaps with variation) in each of many different regions that are scattered throughout the genome.

## Implanted Motif Problem: Find all (k, d)-motifs in a collection of strings
> ### Given a collection of DNA strings and an integer *d*, a k-mer is a (k,d)-motif if it appears in every string from Dna with at most *d* mismatches. 

### Brute force / exhaustive search approach:
> #### Generate all k-mers in the first DNA string, and then check which of them are (k, d)-motifs.

In [ ]:
# Pseudocode

MotifEnumeration(Dna, k, d)
    Patterns ← an empty set
    for each k-mer Pattern in the first string in Dna
        for each k-mer Pattern’ differing from Pattern by at most d mismatches
            if Pattern' appears in each string from Dna with at most d mismatches
                add Pattern' to Patterns
    remove duplicates from Patterns
    return Patterns

In [68]:
def HammingDistance(p, q):
    if len(p) != len(q):
        return -1  # Invalid input, return an error code
    return sum([1 for nucleotide in range(len(p)) if p[nucleotide] != q[nucleotide]])

# Generates all possible neighbors of our pattern with d or fewer mismatches
def Neighbors(pattern, d):
    nucleotides = 'ACGT'
    # Case: d = 0
    if d == 0:
        return {pattern}
    
    # Case: pattern is 1 letter
    if len(pattern) == 1:
        return set(nucleotides)
    
    # Case: d > 0 and pattern > 1 letter
    neighborhood = set()
    # Starting with possible mismatches at the first letter, keeping the suffix of the pattern the same
    # Then, use recursion to iterate over all possible mismatches at each remaining letter index
    suffix_neighbors = Neighbors(pattern[1:], d)
    for neighbor in suffix_neighbors:
        # If the Hamming distace of the suffix is < d, the neighbor differs by at most d mismatches
        if HammingDistance(pattern[1:], neighbor) < int(d):
            for nuc in nucleotides:
                # Concatenate each of the four nucleotides to the suffix of the pattern
                neighborhood.add(nuc + neighbor)
        else:
            neighborhood.add(pattern[0] + neighbor)
    return neighborhood

# This function uses a given number of DNA sequences to check, separated by spaces
def MotifEnumeration(Dna, k, d):
    patterns = set()
    for i in range(len(Dna[0]) - int(k) + 1):
        # Sliding window of length k to find all patterns in all sequences
        pattern = Dna[0][i:i + int(k)]
        # Recursion to find all possible neighbors of each pattern in sliding window
        pattern_neighbors = Neighbors(pattern, d)
        motifs_found = set()
        # For each neighbor in the set of all possible neighbors for all patterns across all sequences
        for neighbor in pattern_neighbors:
            is_motif = True
            # For each sequence in our given number of DNA sequences
            for seq in Dna:
                motif_found = False
                for j in range(len(seq) - int(k) + 1):
                    # If the number of mismatches is d or fewer in a sliding window of patterns
                    if HammingDistance(neighbor, seq[j:j + int(k)]) <= int(d):
                        # Motif found and move to the next pattern in the current sequence
                        motif_found = True
                        break
                if not motif_found:
                    is_motif = False
                    break
            # After all sequences are parsed, parse neighborhood for "is-motif = True"
            # If true, add that motif to "motifs_found" set
            if is_motif:
                motifs_found.add(neighbor)
        patterns.update(motifs_found)
    return ' '.join(list(set(patterns)))



file_path = r'C:\Users\ryanr\OneDrive\Desktop\Coursera\Bioinformatics UCSD\dataset_30302_8.txt'
with open(file_path, 'r') as file:
    K = file.readline().strip()
    D = file.readline().strip()
    # split() creates a list, rather than a line of strings separated by spaces
    DNA = file.readline().strip().split()

# * unpacks elements of the list and prints them separated by spaces
print("Motifs with up to ", D, " mismatches:\n", *MotifEnumeration(DNA, K, D), sep='')

Motifs with up to 1 mismatches:
CCCGC CCCTC CCCCC CCCAC


## Given a k-mer Pattern and a set of strings *Dna = {Dna$_1$, … , Dna$_t$}*, we define *d(Pattern, Dna)* as the sum of distances between *Pattern* and all strings in *Dna*.

> ### For example, for the strings *Dna* shown below, *d(AAA, Dna)* = 1 + 1 + 2 + 0 + 1 = 5.

<img src="d_Pattern_Dna.png" width="500" />

### Our goal is to find a k-mer *Pattern* that minimizes *d(Pattern, Dna)* over all k-mers *Pattern*. We call such a k-mer a **median string** for Dna.

> ### Median String Problem: Find a median string.
 - Input: A collection of strings Dna and an integer k.
 - Output: A k-mer Pattern that minimizes d(Pattern, Dna) among all possible choices of k-mers.

In [ ]:
# Pseudocode for a brute force solution to the Median String Problem

Median_String(Dna, k)
    distance ← ∞
    for each k-mer Pattern from AA…AA to TT…TT
        if distance > d(Pattern, Dna)
             distance ← d(Pattern, Dna)
             Median ← Pattern
    return Median

In [127]:
import itertools

# d(Pattern, Dna)
    # Given a k-mer Pattern and a set of strings Dna:
    # Sum of all minimum Hamming distances between Pattern and all strings Dna
def Sum_Distances(pattern, Dna):
    distance_sum = 0
    k = len(pattern)
    # For each sequence in Dna
    for seq in Dna.split():
        min_distance = float('inf')
        for nuc in range(len(seq) - int(k) + 1):
            # Scanning window of Pattern size
            kmer = seq[nuc:nuc + int(k)]
            # Hamming distance between a given pattern and each window of nucleotides
            distance = HammingDistance(pattern, kmer)
            # Update minimum Hamming distance
            if distance < min_distance:
                min_distance = distance
        distance_sum += min_distance
    return distance_sum

# Find all k-mer Patterns ("median string") that minimize d(Pattern, Dna)
def Median_String(Dna, k):
    min_distance = float('inf')
    medians = []
    # For all possible variations of k nucleotides (pattern)
    for pattern in [''.join(p) for p in itertools.product('ACGT', repeat = int(k))]:
        # Update min_distance generated by Sum_Distances(pattern, Dna) using each pattern
        if Sum_Distances(pattern, Dna) < min_distance:
            min_distance = Sum_Distances(pattern, Dna)
            medians = [pattern]
        elif Sum_Distances(pattern, Dna) == min_distance:
            medians.append(pattern)
    return medians



file_path = r'C:\Users\ryanr\OneDrive\Desktop\Coursera\Bioinformatics UCSD\dataset_30304_9.txt'
with open(file_path, 'r') as file:
    K = file.readline().strip()
    DNA = file.readline().strip()
    
print(f"{K}-mers that minimize d(Pattern, Dna): {Median_String(DNA, K)}")

6-mers that minimize d(Pattern, Dna): ['AACGAT', 'ACAACG', 'ACCAGC', 'ACGATC', 'AGAGTA', 'AGCATA', 'AGTACC', 'ATATGG', 'ATCAAA', 'ATGGTT', 'CAACGA', 'CAGCAT', 'CATATG', 'CCAGCA', 'CGACAA', 'CGATCA', 'CGCGGT', 'CGGTTG', 'GACAAC', 'GAGTAC', 'GATCAA', 'GCATAT', 'GCGACA', 'GCGGTT', 'GGTTGC', 'GGTTTC', 'GTACCA', 'GTTGCG', 'GTTTCG', 'TACCAG', 'TATGGT', 'TCGCGG', 'TGCGAC', 'TGGTTT', 'TTCGCG', 'TTGCGA', 'TTTCGC']


### Shown below is a motif matrix *Motifs* with its nucleotide counts *Count(Motifs)*, profile matrix *Profile(Motifs)* (for which P$_{i,j}$ is the frequency of the i-th nucleotide in the j-th column of the motif matrix), and consensus string *Consensus(Motifs)* (the most popular letters in each column of the motif matrix).

<img src="motifs-count-consensus-profile.png" width="500">

### The probability *Pr(ACGGGGATTACC | Profile)* that *Profile* generates ACGGGGATTACC is computed by simply multiplying the corresponding entries in the profile matrix, shown below:

<img src="profile-prob.png" width="500">

### Given a profile matrix *Profile*, evaluate the probability of every k-mer in a string *Seq* and find a **Profile-most probable** k-mer in *Seq*, i.e., find a k-mer that was most likely to have been generated by *Profile* among all k-mers in *Seq*.

In [126]:
# Find profile-most probable k-mer given a DNA sequence, integer k, and Profile matrix
def Profile_Most_Probable(seq, k, profile):
    max_prob = -1
    most_probable_kmer = ""
    for i in range(len(seq) - int(k) + 1):
        # For each kmer of scanning window size k
        kmer = seq[i:i + int(k)]
        prob = 1
        for nuc in range(int(k)):
            # If current nucleotide == A, C, G or T
            if kmer[nuc] == 'A':
                # Set probability = corresponding probability in Profile
                # As we iterate through to k-th nucleotide in kmer, use that column in Profile
                prob *= profile[0][nuc]
            elif kmer[nuc] == 'C':
                prob *= profile[1][nuc]
            elif kmer[nuc] == 'G':
                prob *= profile[2][nuc]
            elif kmer[nuc] == 'T':
                prob *= profile[3][nuc]
            # kmer now has an associated probability
        if prob > max_prob:
            # Update the maximum probability found and assign the corresponding kmer to moxt_probable_kmer
            max_prob = prob
            most_probable_kmer = kmer
    return most_probable_kmer
    
    
    
file_path = r'C:\Users\ryanr\OneDrive\Desktop\Coursera\Bioinformatics UCSD\dataset_30305_3.txt'
with open(file_path, 'r') as file:
    Seq = file.readline().strip()
    K = file.readline().strip()
    # Read final lines as a matrix
    Profile = [[float(num) for num in line.split()] for line in file]

print(f"Profile-most probable {K}-mer in given sequence: {Profile_Most_Probable(Seq, K, Profile)}")

Profile-most probable 12-mer in given sequence: ATATAAGGCTAA


## Greedy approach to motif finding:

#### **GreedyMotifSearch** will start by forming a motif matrix by selecting the first k-mer in the first *Dna* string. For a given k-mer choice *Motif$_1$* in *Dna$_1$*, it builds a profile matrix for this k-mer, and sets *Motif$_2$* equal to the profile-most probable k-mer in *Dna$_2$*. It then iterates by updating *Profile* to use *Motif$_1$* and *Motif$_2$*, and sets *Motif$_3$* equal to the profile-most probable k-mer in *Dna$_3$*, and so on. After obtaining a k-mer from each string, we test to see whether the current motif matrix *Motifs* outscores the current best scoring one. Finally, *Motif$_1$* moves over one nucleotide in *Dna$_1$* and repeats the whole process.

In [ ]:
# Pseudocode

GreedyMotifSearch(Dna, k, t)
    BestMotifs ← motif matrix formed by first k-mers in each string from Dna
    for each k-mer Motif in the first string from Dna
        Motif1 ← Motif
        for i = 2 to t
            form Profile from motifs Motif1, …, Motifi - 1
            Motifi ← Profile-most probable k-mer in the i-th string in Dna
        Motifs ← (Motif1, …, Motift)
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
    return BestMotifs

In [124]:
# Build a profile matrix Profile for a given k-mer
    # Takes a list of motifs as input
def Build_Profile_Matrix(motifs):
    # Length of each kmer/motif or row in the matrix
    k = len(motifs[0])
    # Initialize profile matrix with 4 rows (A,C,G,T) of lists containing k zeroes
    profile = [[0] * k for _ in range(4)]
    num_rows = len(motifs)
    # For each column
    for nuc in range(k):
        # Initialize dictionary of nucleotide counts
        count = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
        # For each row
        for motif in motifs:
            # Increment nucleotide counts in dictionary
                # motif[nuc] accesses the element of motif at the nuc-th column
                # count[motif[nuc]] is like count['A'] where motif[nuc] becomes the key whose value gets incremented
            count[motif[nuc]] += 1
        # For each row of Profile matrix
        for j in range(4):
            # Get value of j-th key (nucleotide) in {count} and divide by total number of rows in Motifs
                # the Profile rows are consistently 'A,C,G,T' so row 0 corresponds to A, etc.
            profile[j][nuc] = count['ACGT'[j]] / num_rows
    return profile

# Score a motif matrix, where each column is assigned a score and all columns are totaled
# A column's score corresponds to the number of least conserved/most mutated nucleotides across all rows
    # The lower the score, the better / most conserved
def Score(motifs):
    consensus = ""
    # Number of matrix columns
    num_cols = len(motifs[0])
    # Number of matrix rows
    num_rows = len(motifs)
    # For each column
    for nuc in range(num_cols):
        # Initialize dictionary of nucleotide counts, which will update for every column in Motifs
        count = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
        # For each row
        for motif in motifs:
            # Increment each nucleotide in count
            count[motif[nuc]] += 1
        # Build the consensus motif using the most-occurring nucleotide in current column for all rows
        consensus += max(count, key=count.get)
    # Calculate score
    score = 0
    # For each row
    for motif in motifs:
        # For each column
        for nuc in range(num_cols):
            # If the nucleotide differs from the corresponding consensus necleotide, increment score
            if motif[nuc] != consensus[nuc]:
                score += 1
    return score

# Construct motif matrices using Profile, and compare their scores to find the one with the lowest score
    # 't' is number of strings in Dna
def Greedy_Motif_Search(Dna, k, t):
    best_motifs = [string[:int(k)] for string in Dna]
    # Specify/Stay in the first Dna string: building each motif matrix from Motif_1
    for i in range(len(Dna[0]) - int(k) + 1):
        # List of k-mers for each motif matrix, where the first in the list is from the first string Dna_1
        # At first, [kmers] has only Motif_1 from Dna_1, which iterates as a scanning window creating each motif matrix
        kmers = [Dna[0][i:i + int(k)]]
        # For each string in Dna
        for string in range(1, int(t)):
            # Construct Profile using preceding k-mers
            profile = Build_Profile_Matrix(kmers)
            # Append the profile-most probable kmer in the next string
            kmers.append(Profile_Most_Probable(Dna[string], k, profile))
        if Score(kmers) < Score(best_motifs):
            best_motifs = kmers
    return ' '.join(best_motifs)



file_path = r'C:\Users\ryanr\OneDrive\Desktop\Coursera\Bioinformatics UCSD\dataset_30305_5.txt'
with open(file_path, 'r') as file:
    K = file.readline().strip()
    T = file.readline().strip()
    # Read space-separated strings as a list
    DNA = file.readline().strip().split()

print("Optimal motif matrix from Dna using greedy approach:\n", Greedy_Motif_Search(DNA, K, T), sep='')

Optimal motif matrix from Dna using greedy approach:
CACACCGAGCTG ACCCTCCACATC TTAGTCCGAAGC CCTCAGTAGATT CCTCAGTAAGAA CCTCAGCAAGAA CTCCCCTAGATA CACAAGCGGGAT GAGTGGAGGGAC TCACTGTAGGTC TTTCAGCAGCAT CTTTAGTAAGAA GATTAGCAAGAA CATCAGTAAGAA CTTAAGGAAGAA TTTGAGGAAGAA ATTCAGTAAGAA GTTGAGTAAGAA TTTTAGTAAGAA TCTGAGAAAGAA CTACAGGGGGTT GATAAGCAAGAA AATTAGCAAGAA TCTGAGTAAGAA CTTCAGCAAGAA


### The above **GreedyMotifSearch** algorithm has a crucial flaw, which is the pervasive presence of zeroes in each *Profile* matrix. After finding the first *Profile* matrix for *Motif$_1$* and searching for the subsequent profile-most probable *Motif$_2$*, the probability of every k-mer except *Motif$_1$* is zero.

> #### Cromwell's rule!

### A solution is to use Laplace's Rule of Succession and simply add 1 to each element of *Count(Motifs)*
> #### Using pseudocounts instead of zeroes

In [125]:
# GreedyMotifSearch with pseudocounts using Laplace's Rule of Succession

# Build a profile matrix Profile for a given k-mer
    # Takes a list of motifs as input
def Build_Profile_Matrix_New(motifs):
    # Length of each kmer/motif or row in the matrix
    k = len(motifs[0])
    # Initialize profile matrix with 4 rows (A,C,G,T) of lists containing k zeroes
    profile = [[0] * k for _ in range(4)]
    num_rows = len(motifs)
    # For each column
    for nuc in range(k):
        # Initialize dictionary of nucleotide counts
        # LAPLACE'S RULE OF SUCCESSION: This is the only line we need to change
        count = {'A': 1, 'C': 1, 'G': 1, 'T': 1}
        # For each row
        for motif in motifs:
            # Increment nucleotide counts in dictionary
                # motif[nuc] accesses the element of motif at the nuc-th column
                # count[motif[nuc]] is like count['A'] where motif[nuc] becomes the key whose value gets incremented
            count[motif[nuc]] += 1
        # For each row of Profile matrix
        for j in range(4):
            # Get value of j-th key (nucleotide) in {count} and divide by total number of rows in Motifs
                # the Profile rows are consistently 'A,C,G,T' so row 0 corresponds to A, etc.
            profile[j][nuc] = count['ACGT'[j]] / num_rows
    return profile

def Greedy_Motif_Search(Dna, k, t):
    best_motifs = [string[:int(k)] for string in Dna]
    # Specify/Stay in the first Dna string: building each motif matrix from Motif_1
    for i in range(len(Dna[0]) - int(k) + 1):
        # List of k-mers for each motif matrix, where the first in the list is from the first string Dna_1
        # At first, [kmers] has only Motif_1 from Dna_1, which iterates as a scanning window creating each motif matrix
        kmers = [Dna[0][i:i + int(k)]]
        # For each string in Dna
        for string in range(1, int(t)):
            # Construct Profile using preceding k-mers
            profile = Build_Profile_Matrix_New(kmers)
            # Append the profile-most probable kmer in the next string
            kmers.append(Profile_Most_Probable(Dna[string], k, profile))
        if Score(kmers) < Score(best_motifs):
            best_motifs = kmers
    return ' '.join(best_motifs)



file_path = r'C:\Users\ryanr\OneDrive\Desktop\Coursera\Bioinformatics UCSD\dataset_30306_9.txt'
with open(file_path, 'r') as file:
    K = file.readline().strip()
    T = file.readline().strip()
    # Read space-separated strings as a list
    DNA = file.readline().strip().split()

print("Optimal motif matrix from Dna:\n", Greedy_Motif_Search(DNA, K, T), sep='')

Optimal motif matrix from Dna:
CAAGCTTTCCTA CTACCTTTCCTT CAAACTTACCTG CGATCTTACCTA CTAGCTTGCCTT CGATCTTCCCTC CGAACTTTCCTT CAAACTTTCCTA CTACCTTCCCTC CGACCTTACCTT CAAACTTCCCTG CAAGCTTACCTC CAAGCTTACCTT CGAGCTTGCCTA CCAACTTGCCTT CCACCTTACCTA CGAGCTTGCCTT CAACCTTTCCTC CGAGCTTACCTA CCATCTTTCCTG CCACCTTCCCTT CGACCTTGCCTC CGACCTTTCCTC CAAACTTGCCTC CGATCTTGCCTT


In [122]:
import math

# Calculate the entropy of a list of numbers rounded to 2 decimal points
def CalculateEntropy(numbers):
    ans = 0
    for num in numbers:
        if num == 0:
            # Creating rule that log_2(0) = 0
            ans -= 0
        else:
            ans -= num * math.log2(num)
    return round(ans, 2)
    
    
    
numbers_1 = [0.5, 0, 0, 0.65]
numbers_2 = [0.2, 0.25, 0.3, 0.25]
numbers_3 = [0, 0, 0.9, 1]
numbers_4 = [0.25, 0, 0.7, 0.25]
numbers_5 = [0.2, 0.6, 0.0, 0.2]

# Call function iteratively using eval()
for i in range(5):
    numbers_i = eval(f"numbers_{i+1}")
    print(CalculateEntropy(numbers_i))

0.9
1.99
0.14
1.36
1.37
